Bárbara Duarte

Este primeiro script faz uma consulta via API no Binance, empresa referencia para cotação de cryptomoedas, cotação de bolsas de valores, ações, entre outros. O script tem a função de exibir em display, que é atualizado por segundo, a cotação das moedas presentes em uma lista pré determinada e fornece um botão para gerar um csv no momento exato da captura das moedas.

In [41]:
import tkinter as tk
from tkinter import messagebox
import requests
import csv

In [42]:
url = "https://api.binance.com/api/v3/ticker/price"
cryptos = ["BTC", "ETH", "POL", "SOL", "XRP","LTC"]
caminho_arquivo = "cotacao_criptomoedas.csv" #Deixei sem caminho para nao ter problema de permissao em diretorio ao exportar
dcr_crypto = {}

In [43]:
# Função para obter o valor das moedas
def obter_cotacao(crypto):
    try:
        response = requests.get(url, params={"symbol": crypto})
        response.raise_for_status()
        data = response.json()
        return float(data['price'])
    except requests.exceptions.RequestException as e:
        messagebox.showerror("Erro na requisição", f"Erro ao obter valores para {crypto}: {str(e)}")
        root.quit()
        return None


# Função para atualizar os preços
def atualizar_cotacao():
    for crypto in cryptos:
        preco_brl = obter_cotacao(f"{crypto}BRL")
        preco_usd = obter_cotacao(f"{crypto}USDT")

        if preco_brl is not None and preco_usd is not None:
            dcr_crypto[crypto]["brl"].config(text=f"R$ {preco_brl:.2f}")
            dcr_crypto[crypto]["usd"].config(text=f"$ {preco_usd:.2f}")

    # Taxa de atualização dos valores por segundo
    root.after(1000, atualizar_cotacao)


# Função para gerar o arquivo CSV
def gerar_csv():
    with open(caminho_arquivo, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Criptomoeda", "Preço (BRL)", "Preço (USD)"])

        for crypto in cryptos:
            preco_brl = dcr_crypto[crypto]["brl"].cget("text").replace("R$ ", "")
            preco_usd = dcr_crypto[crypto]["usd"].cget("text").replace("$ ", "")
            writer.writerow([crypto, preco_brl, preco_usd])

    messagebox.showinfo("Sucesso", "Arquivo salvo")


# Função principal 
def main():
    global root
    root = tk.Tk()
    root.title("Tabela de Preços Criptomoedas")
    root.geometry("400x300")

    tk.Label(root, text="Criptomoeda", font=("Arial", 12, "bold")).grid(row=0, column=0, padx=10, pady=5)
    tk.Label(root, text="Preço (BRL)", font=("Arial", 12, "bold")).grid(row=0, column=1, padx=10, pady=5)
    tk.Label(root, text="Preço (USD)", font=("Arial", 12, "bold")).grid(row=0, column=2, padx=10, pady=5)

    # Criando os labels para cada criptomoeda
    for i, crypto in enumerate(cryptos, start=1):
        tk.Label(root, text=crypto, font=("Arial", 11)).grid(row=i, column=0, padx=10, pady=5)

        dcr_crypto[crypto] = {
            "brl": tk.Label(root, text="Carregando...", font=("Arial", 11)),
            "usd": tk.Label(root, text="Carregando...", font=("Arial", 11))
        }

        dcr_crypto[crypto]["brl"].grid(row=i, column=1, padx=10, pady=5)
        dcr_crypto[crypto]["usd"].grid(row=i, column=2, padx=10, pady=5)

    # Botão para gerar CSV
    tk.Button(root, text="Gerar CSV", command=gerar_csv, font=("Arial", 12, "bold")).grid(
        row=len(cryptos) + 1, column=0, columnspan=3, pady=10
    )

    atualizar_cotacao()
    root.mainloop()

if __name__ == "__main__":
    main()

Este segundo script realiza o mesmo procedimento, porem gera apenas um csv com os dados das moedas selecionadas

In [35]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import csv
import time

In [36]:
cryptos = ["BTC", "ETH", "POL", "SOL", "XRP","LTC"]
caminho_arquivo = "cotacao_criptomoedas.csv" #Deixei sem caminho para nao ter problema de perissao em diretorio ao salvar o arquivo 
erro_obterCotacao = "Não foi possível obter cotacao"
erro_convertCotacao = "Não foi possível converter o valor para float"

In [37]:
options = Options()
options.headless = True  

# Função para limpar a string que armazena a cotação
def limpar_cotacao(cotacao):
    return cotacao.strip().replace('$', '').replace('R$', '').replace(',', '')


# Função para obter cotação da moeda
def obter_cotacao(crypto):
    url_usd = f"https://www.binance.com/en/trade/{crypto}_USDT"
    url_brl = f"https://www.binance.com/en/trade/{crypto}_BRL"

    driver = webdriver.Chrome(options=options)

    cotacao_usd = None
    cotacao_brl = None

    # Obtém cotação em USD
    driver.get(url_usd)
    carregar_janela = 0
    while carregar_janela < 60:
        try:
            cotacao_elemento_usd = driver.find_element(By.CLASS_NAME, "subPrice")
            cotacao_usd = limpar_cotacao(cotacao_elemento_usd.text)
            break
        except:
            carregar_janela += 1
            time.sleep(1)

    # Obtém cotação em BRL
    driver.get(url_brl)
    carregar_janela = 0
    while carregar_janela < 60:
        try:
            cotacao_elemento_brl = driver.find_element(By.CLASS_NAME, "subPrice")
            cotacao_brl = limpar_cotacao(cotacao_elemento_brl.text)
            break
        except:
            carregar_janela += 1
            time.sleep(1)

    driver.quit()

    if cotacao_usd is None or cotacao_brl is None:
        return erro_obterCotacao
    else:
        try:
            return float(cotacao_usd), float(cotacao_brl)
        except ValueError:
            return erro_convertCotacao


# Função para escrever no arquivo CSV
def escrever_csv(dados):
    with open(caminho_arquivo, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Criptomoeda", "Preço (USD)", "Preço (BRL)"])
        writer.writerows(dados)

    print(f"Arquivo {caminho_arquivo} exportado!")


# Função principal
def main():
    dados = []
    for crypto in cryptos:
        cotacao = obter_cotacao(crypto)
        if cotacao == erro_obterCotacao or cotacao == erro_convertCotacao:
            dados.append([crypto, cotacao, cotacao])
        else:
            cotacao_usd, cotacao_brl = cotacao
            dados.append([crypto, f"$ {cotacao_usd:.2f}", f"R$ {cotacao_brl:.2f}"])

    escrever_csv(dados)


# Executa a main()
if __name__ == "__main__":
    main()

Arquivo cotacao_criptomoedas.csv exportado!
